In [84]:
# imports and set API url
import requests
import pandas as pd
import numpy as np
import time

url = 'https://api.pushshift.io/reddit/search/submission'     

In [87]:
# create a function to aggregate 10_000 posts per subreddit

def get_posts_retail(url):
    #set date
    before_date = 1621889311 #change this to "today"
    all_posts = pd.DataFrame()
    # loop through 100 posts in a given period to get 1000 posts per subreddit
    for sub in ['TalesFromRetail', 'talesfromtechsupport']:
        for i in range(100):
            params = {
                'subreddit'   : sub,
                'size'        : 100,
                'pinned'      : False, 
                'selftext:not': '[removed]',
                'before'      : before_date
                }  

            res = requests.get(url, params)
            
            try:
                res.raise_for_status() 
            except:
                print('Bad Request: ', res.status_code, i, before_date)
                
            if 'json' in res.headers.get('Content-Type'):
                data = res.json()
                posts = data['data']
                posts_df = pd.DataFrame(posts)[['subreddit', 'title', 'selftext', 'author', 'created_utc']]       
                before_date = posts_df['created_utc'].min()
                all_posts = all_posts.append(posts_df, ignore_index=True)
            else:
                print('Response content is not in JSON format.')
                return res
            if i%10 == 0:
                print(f'{i} Iteration')
            time.sleep(1) # give time between requests  
                                
    return all_posts    

In [88]:
# run function
df = get_posts_retail(url)

0 Iteration
10 Iteration
20 Iteration
30 Iteration
40 Iteration
50 Iteration
60 Iteration
70 Iteration
80 Iteration
90 Iteration
0 Iteration
10 Iteration
20 Iteration
30 Iteration
40 Iteration
50 Iteration
60 Iteration
70 Iteration
80 Iteration
90 Iteration


In [91]:
# drop na's
df.dropna(inplace = True)

In [94]:
# export to csv
df.to_csv('data/posts.csv')